In [ ]:
from prettyprinter import pprint
import polars as pl
import asyncio
import httpx


In [ ]:
series_ids = [
    "STEO.RGDPQ_NONOECD.M",
    "STEO.RGDPQ_OECD.M",
    "STEO.PAPR_NONOPEC.M",
    "STEO.PAPR_OPEC.M",
    "STEO.PATC_OECD.M",
    "STEO.PATC_NON_OECD.M",
    "STEO.COPRPUS.M",
    "STEO.CORIPUS.M",
    "STEO.FOREX_WORLD.M",
    "STEO.PASC_OECD_T3.M",
    "STEO.COPS_OPEC.M",
    "STEO.COPC_OPEC.M",
    "STEO.T3_STCHANGE_OOECD.M",
    "STEO.T3_STCHANGE_NOECD.M"
]

X_Params = {
    "frequency": "monthly",
    "data": [
        "value"
    ],
    "facets": {},
    "start": "2000-01",
    "end": "2024-01",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
    "offset": 0,

}

API_KEY = "caGyhsgCC3brWOYqKDFMtPc16VPABFzHr3PIfHf7"

In [ ]:
async def fetch_eia_data(series_id: str, headers: dict = {}):
    url = f"https://api.eia.gov/v2/seriesid/{series_id}?api_key={API_KEY}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        response.raise_for_status()  
        return response.json()


async def get_all_data (list_of_series_id: list, headers: dict = {}):
    tasks = [fetch_eia_data(series_id, headers) for series_id in list_of_series_id]
    return await asyncio.gather(*tasks)


responses = await get_all_data(list_of_series_id=series_ids,headers=X_Params)
DATA = list(map(lambda x: pl.DataFrame(x['response']['data']), filter(lambda x: len(x['response']['data']), responses)))

In [ ]:
metadata = {
    "series_id": list(map(lambda x: x['seriesId'][0], DATA)),
    "description": list(map(lambda x: x['seriesDescription'][0], DATA)),
    "unit": list(map(lambda x: x['unit'][0], DATA)),
    "n_observations": list(map(lambda x: x.shape[0], DATA)),
    }


In [ ]:
print(pl.DataFrame(metadata).to_pandas().to_markdown(index=False))

In [ ]:
import pickle
with open("DATA.pkl", "wb") as f:
    pickle.dump(list(map(lambda x: x.to_pandas(), DATA)), f)

In [ ]:

# First plot
multivariate_density(x=DATA[0]["value"], y=DATA[1]["value"], 
                     x_label="GDP Non-OECD", y_label="GDP OECD", title="Supply Non-OECD vs Supply OECD")

# Second plot
multivariate_density(x=DATA[2]["value"], y=DATA[3]["value"], 
                     x_label=metadata['description'][2], y_label=metadata['description'][3], title="Consumption Non-OECD vs Consumption OECD")




In [2]:
from utils import get_data as gd
oil = gd.TimeSeries("CL=F")

[*********************100%%**********************]  1 of 1 completed


In [21]:
def filter_uppercase(s):
    return ''.join([c for c in s if c.isupper()])

HW


In [25]:
import os 
import polars as pl
from functools import reduce
macro_data = list(filter(lambda x: x.startswith("Political"), os.listdir("macro_data")))

def reduce_join (list_of_data: list, on: str):
    return reduce(lambda x,y: x.join(y, on=on, how="inner"), list_of_data)

political_stability_data = reduce_join([pl.read_csv(f"macro_data/{data}").rename({"value": filter_uppercase(data)}) for data in macro_data], "period")
# 

period,PSIAGO,PSIARE,PSIBRA,PSICAN,PSICHN,PSICOL,PSIDZA,PSIEGY,PSIGAB,PSIGNQ,PSIIRN,PSIIRQ,PSIKAZ,PSIKWT,PSILBN,PSILBR,PSILBY,PSILCA,PSIMEX,PSIMYS,PSINGA,PSINLD,PSINOR,PSIOMN,PSIQAT,PSIRUS,PSISAU,PSISDN,PSISYR,PSITKM,PSITTO,PSITUN,PSIUSA,PSIUZB,PSIVEN,PSIYEM
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1996-01-01""",-2.061917,0.964015,-0.22046,1.234052,-0.097849,-1.647859,-1.783311,-0.526032,0.125683,-0.131352,-0.324624,-1.817377,-0.367461,0.170836,-0.668953,-2.439463,-0.988059,1.035551,-0.925459,0.57122,-1.056908,1.526126,1.456822,0.873983,0.361695,-1.177854,-0.189458,-2.479532,-0.34796,0.449251,0.273877,0.262547,0.935006,-0.543732,-0.584048,-1.311538
"""1998-01-01""",-2.318058,0.871309,-0.339506,0.950881,-0.394105,-1.793922,-1.878905,-0.02077,0.325021,-0.080598,-0.693391,-1.5338,0.194384,0.455113,-0.752315,-1.755551,-0.80974,0.350062,-0.423062,-0.038123,-0.588244,1.594803,1.439661,0.854953,1.058855,-1.089741,0.110229,-2.259968,-0.044107,0.197645,0.412276,0.263467,0.879613,-0.683049,-0.537717,-1.192847
"""2000-01-01""",-2.038174,0.975434,0.184102,1.167527,-0.209917,-1.608961,-1.432577,0.049798,0.640211,-0.035039,-0.727548,-1.732643,0.09009,0.752233,-0.439045,-1.952881,-0.294287,1.2236,-0.20021,0.092691,-1.455326,1.758681,1.533283,1.121425,1.157301,-1.399242,0.227822,-2.244173,-0.180451,0.008447,0.153456,0.321115,1.082809,-1.30195,-0.758917,-1.145441
"""2002-01-01""",-1.57618,0.937275,0.3278,1.218032,-0.327627,-1.956714,-1.634565,-0.414172,0.410249,-0.514396,-0.773441,-1.601287,0.358549,-0.146744,-0.354815,-2.18962,-0.095381,0.362932,-0.058308,0.539594,-1.626329,1.30268,1.608346,0.928577,0.831209,-0.724512,0.002424,-1.846431,0.277536,0.070418,-0.094292,0.154226,0.285477,-1.056884,-1.317897,-1.30688
"""2003-01-01""",-1.007386,0.99465,0.004214,1.053077,-0.565358,-2.376027,-1.754409,-0.640925,0.299039,0.114631,-0.779733,-2.393245,0.35626,0.294287,-0.459643,-2.159393,0.120704,1.169785,-0.116878,0.485535,-1.635588,1.161806,1.204556,1.061506,1.186481,-1.193152,0.141018,-1.977056,0.063645,0.053216,-0.250032,0.317871,0.08044,-1.299195,-1.286445,-1.481762
"""2004-01-01""",-1.063121,0.768758,-0.275993,0.876447,-0.390681,-2.271226,-1.359188,-0.822402,0.408171,-0.059478,-0.761824,-3.180352,0.070388,0.34524,-0.713361,-1.365386,0.355681,1.218339,-0.214082,0.348004,-1.754416,1.016239,1.196618,1.115968,1.123905,-1.514594,-0.654633,-1.549037,-0.285746,-0.115212,-0.068327,0.148958,-0.23304,-1.455517,-1.391586,-1.551187
"""2005-01-01""",-0.885178,0.875413,-0.26338,0.828725,-0.501757,-2.057943,-0.913666,-0.629359,0.325626,-0.386099,-0.782739,-2.688648,0.190632,0.25369,-0.999741,-1.357909,0.448557,1.000115,-0.445821,0.566449,-1.6665,0.988101,1.295277,0.946495,1.027679,-1.267701,-0.240614,-1.973296,-0.461609,0.072713,-0.121956,0.024227,-0.060543,-1.957896,-1.244332,-1.436673
"""2006-01-01""",-0.539453,0.923421,-0.294911,1.034957,-0.54003,-1.904797,-1.126413,-0.837808,0.220006,0.16879,-1.04084,-2.826402,0.207938,0.412733,-1.803863,-1.326841,0.358129,0.860106,-0.636888,0.28838,-2.034078,0.902736,1.252706,0.841324,0.926922,-0.908794,-0.522424,-2.139077,-0.278218,-0.244654,-0.237762,0.208512,0.490835,-1.78466,-1.252868,-1.345312
"""2007-01-01""",-0.666659,0.975391,-0.357948,1.007011,-0.500554,-1.806527,-1.150586,-0.552382,0.302845,0.256655,-0.93164,-2.771051,0.647997,0.620644,-2.119095,-1.24484,0.747058,0.617986,-0.729852,0.193307,-2.014056,0.778469,1.145553,0.939814,0.952528,-0.863383,-0.467305,-2.362725,-0.339876,0.277233,-0.188574,0.158262,0.374478,-1.362033,-1.22967,-1.594252


In [4]:
oil.data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,54.200001,55.240002,52.110001,52.330002,52.330002,727793
1,2017-01-04,52.490002,53.430000,52.150002,53.259998,53.259998,512641
2,2017-01-05,53.389999,54.119999,52.790001,53.759998,53.759998,517362
3,2017-01-06,53.730000,54.320000,53.320000,53.990002,53.990002,528333
4,2017-01-09,53.750000,53.830002,51.759998,51.959999,51.959999,564893
...,...,...,...,...,...,...,...
1886,2024-07-02,83.430000,84.379997,82.720001,82.809998,82.809998,367787
1887,2024-07-03,83.199997,83.930000,82.459999,83.879997,83.879997,289685
1888,2024-07-05,83.599998,84.519997,83.019997,83.160004,83.160004,361357
1889,2024-07-08,83.150002,83.320000,82.080002,82.330002,82.330002,361357
